In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall dill
!pip install dill==0.3.4

Found existing installation: dill 0.3.6
Uninstalling dill-0.3.6:
  Would remove:
    /usr/local/bin/get_objgraph
    /usr/local/bin/undill
    /usr/local/lib/python3.8/dist-packages/dill-0.3.6.dist-info/*
    /usr/local/lib/python3.8/dist-packages/dill/*
Proceed (y/n)? y
  Successfully uninstalled dill-0.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.
evaluate 0.4.0 requires datasets>=2.0.0, but you have datasets 1.0.2 which is incompatible.


## Train

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 31.1 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645950 sha256=d0029ce5fe9029193e45e4235840a1ca413e3beabb0057c94ea74238b220d80e
  Stored in directory: /root/.cache/pip/wheels/3c/e9/86/706911c12e307aeb9a2702399f0dad38d36f1d6f9dde8af35e
Successfully built vncorenlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.4 MB/s 
  Attempting uninstall: datasets
    Found existing installation: datasets 2.7.1
    Uninstalling datasets-2.7.1:
      Successfully uninstalled datasets-2.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
evaluate 0.4.0 requires datasets>=2.0.0, but you have data

In [ ]:
!gsutil cp -r gs://vietai_public/viT5/data/vietnews .
!gsutil cp -r gs://vietai_public/viT5/data/wikilingua .

Copying gs://vietai_public/viT5/data/vietnews/test.tsv...
Copying gs://vietai_public/viT5/data/vietnews/test_dedup.tsv...
Copying gs://vietai_public/viT5/data/vietnews/train_dedup.tsv...
| [3 files][422.4 MiB/422.4 MiB]                                                
Operation completed over 3 objects/422.4 MiB.                                    
Copying gs://vietai_public/viT5/data/wikilingua/test.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/train.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/val.tsv...
\ [3 files][ 63.5 MiB/ 63.5 MiB]                                                
Operation completed over 3 objects/63.5 MiB.                                     


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments, AutoModel
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")  
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/bartpho-word")

#tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
#model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")


model.to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64001, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64001, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )
    
    labels = tokenizer(
        examples["labels"], max_length=64, truncation=True, padding=True
    )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
#data = pd.read_excel("/content/drive/MyDrive/D:  /Đại học/Năm 3/Kỳ 1/Xử lí ngôn ngữ tự nhiên/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/data_Tuoi_Tre.xlsx")
#data = pd.read_excel("/content/drive/MyDrive/data_Tuoi_Tre.xlsx")

train = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/train.xlsx")
val = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/val.xlsx")
test = pd.read_excel("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Data/Data4model/test.xlsx")

In [ ]:
#data = data.iloc[:100, :]

In [ ]:
input_lines = train["Content"].values
label_lines = train["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
train_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
input_lines = val["Content"].values
label_lines = val["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
val_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
input_lines = test["Content"].values
label_lines = test["Summary"].values

for i in range(len(input_lines)):
  input_lines[i] = str(input_lines[i]) + '</s>'

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3",
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=3,
                                      learning_rate=1e-4,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=2,
                                      per_device_eval_batch_size=2,
                                      logging_dir='./log',
                                      group_by_length=True,
                                      save_strategy="steps",
                                      save_steps= 2000,  
                                      save_total_limit=10,
                                      overwrite_output_dir=True,
                                      fp16=True,
                                      )
# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37

# from transformers.optimization import Adafactor, AdafactorSchedule
# optimizer = Adafactor(
#     model.parameters(),
#     lr=1e-3,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False
# )
# lr_scheduler = AdafactorSchedule(optimizer)


In [ ]:
import torch
torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=_traintokenized_datasets,
    data_collator=data_collator,
    eval_dataset= val_tokenized_datasets,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 29893
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 44841
  Number of trainable parameters = 420361216


Step,Training Loss
500,3.155300
1000,2.296100
1500,2.241400
2000,2.342600
2500,2.482400
3000,2.461600
3500,2.415800
4000,2.395000
4500,2.368200
5000,2.364100


Saving model checkpoint to /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-2000
Configuration saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-4000
Configuration saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-4000/config.json
Model weights saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Mode

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-8edd18974a53>", line 11, in <module>
    trainer.train()
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1527, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1837, in _inner_training_loop
    self.scaler.step(self.optimizer)
  File "/usr/local/lib/python3.8/dist-packages/torch/cuda/amp/grad_scaler.py", line 341, in step
    retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/cuda/amp/grad_scaler.py", line 288, in _maybe_opt_step
    retval = optimizer.step(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py", line 68, in wrapper
    return wrapped(*args, **

KeyboardInterrupt: ignored

In [ ]:
import torch
#torch.cuda.empty_cache()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler)
)
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-40000")

Using cuda_amp half precision backend
Loading model from /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-40000.
The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 29893
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps

  0%|          | 0/10106 [00:00<?, ?it/s]

Step,Training Loss
40500,0.989300
41000,0.992000
41500,0.989100
42000,0.960100
42500,0.998300
43000,0.958700
43500,0.985900
44000,0.993400
44500,0.970300


Saving model checkpoint to /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-42000
Configuration saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-42000/config.json
Model weights saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-42000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-44000
Configuration saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-44000/config.json
Model weights saved in /content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-44000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=44841, training_loss=0.10606480476350641, metrics={'train_runtime': 2808.1344, 'train_samples_per_second': 31.935, 'train_steps_per_second': 15.968, 'total_flos': 1.9434603045873254e+17, 'train_loss': 0.10606480476350641, 'epoch': 3.0})

## Inference

In [ ]:
!mv /content/temp/checkpoint-10000 /content/drive/MyDrive

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")


Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [ ]:
input_lines = []
label_lines = []
task = 'wikilingua'
with open(f'{task}/test.tsv') as file:
  for line in file:
    line = line.strip().split('\t')
    input = line[0]
    input_lines.append(input +'</s>')
    label_lines.append(line[1])



input_lines  = input_lines
label_lines = label_lines
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Đồ án DS310 - Xử lí ngôn ngữ tự nhiên/Model/BARTPho-word/epoch3/checkpoint-44000")
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/checkpoint-10000")  
model.to('cuda')

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64001, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64001, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor

In [ ]:
import torch 
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(val_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()


  0%|          | 0/117 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.6115088236707809, recall=0.5651223822690056, fmeasure=0.5746260455883623), mid=Score(precision=0.6158903243248348, recall=0.5696141528927743, fmeasure=0.5778855941592502), high=Score(precision=0.620131426294889, recall=0.5743201136546836, fmeasure=0.5814044385284818)),
 'rougeL': AggregateScore(low=Score(precision=0.3872853138224935, recall=0.35798774283054324, fmeasure=0.3635503316187511), mid=Score(precision=0.3913958252796764, recall=0.36213203770188096, fmeasure=0.3672539776842556), high=Score(precision=0.3958727796484118, recall=0.3663990367748857, fmeasure=0.37128459333336883))}

In [ ]:
rouge_output = metrics.compute(predictions=predictions, references=references, rouge_types=["rouge1","rouge2","rougeL"])
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.6158903243248348, recall=0.5696141528927743, fmeasure=0.5778855941592502)
rouge2
Score(precision=0.27076173724492425, recall=0.25125542774791243, fmeasure=0.2545314679449651)
rougeL
Score(precision=0.3913532458655102, recall=0.3623553705549759, fmeasure=0.36743325780149916)


In [ ]:
for i in range(1,11):  
  print('Prediction: ',predictions[i])
  print('Truth: ',label_lines[i])
  print('Content: ',input_lines[i])

Prediction:  UBND tỉnh Bình Phước vừa có văn bản chỉ đạo các địa phương không được chủ quan, không để người dân tiếp tục đăng ký hoặc thông tin sai sự thật, gây hoang mang trong dư luận.
Truth:  Chọn dụng cụ trực quan giúp ích cho khán giả. Dùng dụng cụ trực quan phù hợp với bài phát biểu. Cẩn trọng khi sử dụng phần mềm trình chiếu Powerpoint.
Content:  Có nhiều lý do để bạn dùng dụng cụ trực quan. Chúng sẽ giúp cho những gì bạn đang truyền đạt trở nên dễ hiểu hơn, giúp khán giả nhớ được những gì bạn nói, thu hút những người học bằng trực quan và làm cho bài nói của bạn trở nên thuyết phục hơn. Hãy nắm rõ mục đích bạn sử dụng từng dụng cụ trực quan trong bài nói của mình. Việc sử dụng dụng cụ trực quan rất hữu ích, tuy nhiên bạn cần chọn dùng những dụng cụ phù hợp. Ví dụ, trong bài nói về kim cương ở trên, nếu muốn khán giả của mình biết về bốn tiêu chí cần xem xét khi chọn mua kim cương, bạn nên trình chiếu một biểu đồ minh họa về nơi người thợ đá quý cắt viên kim cương, hoặc có thể t

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "0"

In [ ]:
sentence = """
Chương trình hành động của Chính phủ đã đề ra 33 nhiệm vụ tập trung vào 5 nhóm nhiệm vụ giải pháp, trước hết là nhóm nhiệm vụ nâng cao, thống nhất nhận thức về đặc thù của đô thị, vai trò, vị thế của đô thị trong sự phát triển chung. Trong đó, xác định phát triển đô thị gồm 3 trụ cột chính là quy hoạch, xây dựng và quản lý đô thị.
"Quy hoạch phải đi trước một bước với tư duy đột phá, tầm nhìn chiến lược. Quy hoạch lúng túng, chậm chạp, không được đầu tư ngang tầm thì lãng phí nguồn lực, phát triển không bền vững, phát triển không đột phá. Công tác quy hoạch làm tốt thì mới tiết kiệm được nguồn lực, xác định và phát huy được tiềm năng khác biệt, cơ hội nổi trội, lợi thế cạnh tranh, chỉ ra và hóa giải những khó khăn, thách thức của địa phương, của vùng”, Thủ tướng phân tích.
Theo Thủ tướng, quy hoạch tổng thể nhưng thực hiện có thể phân kỳ phù hợp với nguồn lực; có thể kiên trì thực hiện trong 5 năm, 10 năm, 20 năm, thậm chí hàng trăm năm… và nếu tôn trọng, làm theo quy hoạch hoàn chỉnh thì sẽ có một đô thị trật tự và phát triển.
Ngoài ra cần đầu tư phát triển hệ thống hạ tầng đô thị đồng bộ, hiện đại trên cơ sở huy động mạnh mẽ các nguồn lực, có cơ chế, chính sách để huy động nguồn lực, kết hợp hài hòa, hợp lý, hiệu quả giữa nguồn lực bên trong và bên ngoài, giữa nguồn lực Nhà nước với các nguồn lực xã hội; đồng thời tiết kiệm, tăng thu, giảm chi, phân bổ và sử dụng nguồn lực hiệu quả, đầu tư có trọng tâm, trọng điểm.
“Hiện vẫn còn nhiều vướng mắc về thể chế, cơ chế, chính sách cần xử lý nhưng với cơ chế đang có thì các địa phương vẫn có thể làm được nếu lãnh đạo các bộ, ngành, địa phương tập trung suy nghĩ, vận dụng tối đa các quy định hiện có trên tinh thần dám nghĩ, dám làm", Thủ tướng lưu ý.
Bùi Thạc Chuyên cho biết gặp áp lực khi chuyển thể văn học từ các tác phẩm của Nguyễn Ngọc Tư. Theo anh, truyện của nữ nhà văn thường có lối viết ngắn nhưng giàu tính gợi mở, các nhân vật có số phận, chứa đựng nội tâm và tiếng nói mạnh mẽ. Do đó, ngoài yếu tố sáng tạo, anh cố gắng để giữ lại những gì mình tâm đắc trong truyện gốc. 
Tro tàn rực rỡ có sự tham gia của Phương Anh Đào, Lê Công Hoàng, Bảo Ngọc Doling, Ngô Quang Tuấn, Hạnh Thúy… Phim là câu chuyện của ba phụ nữ với 3 số phận khác nhau, cách họ sống với niềm vui, nỗi ẩn ức và ngọn lửa tình yêu với chính những người đàn ông của đời mình. Bối cảnh của chuyện xảy ra ở xóm Thơm Rơm, Cà Mau với nhiều chất liệu điện ảnh và văn hóa.
Chia sẻ với VietNamNet, Phương Anh Đào cho biết đây là vai diễn thử thách lớn nhất trong sự nghiệp diễn xuất. Trong phim, diễn viên đảm nhận vai Nhàn - một trong 3 nữ chính. Nhàn có số phận tréo ngoe, luôn khao khát yêu và được yêu đến cháy bỏng. Vốn được biết đến với nét đẹp hiện đại, song nữ diễn viên đã “lột xác” ấn tượng với vai diễn người phụ nữ khắc khổ, tảo tần trong phim.
“Tôi thay đổi hình ảnh rất nhiều cho vai diễn với mong muốn mang đến một Phương Anh Đào khác trên màn ảnh. Thay vì những cảnh bầm dập thể xác, nhân vật của tôi chứa nhiều thương tổn tinh thần. Mọi cảm xúc đều được giữ ở bên trong, và nó khiến nhiều người suy ngẫm. Đó cũng là cách kể chuyện của nhân vật mà anh Bùi Thạc Chuyên muốn hướng đến trong phim”, cô chia sẻ. 
Phim ghi nhận điểm sáng với diễn xuất của các diễn viên. Họ lột tả được nhân vật của mình khi bị đẩy vào hoàn cảnh cùng cực và cách thích nghi một cách bị động trước cuộc đời gần như không lối thoát. 
Hai diễn viên nam Lê Công Hoàng và Quang Tuấn thể hiện tròn vai giữa bối cảnh câu chuyện hầu hết đều xoay quanh những người đàn bà. Diễn viên Quang Tuấn tiết chế được biểu cảm cường điệu so với các vai diễn trước đây. Anh khắc họa một gã đàn ông đau đớn mất con gái và nỗi ám ảnh thường trực vì ánh sáng rực rỡ của những ngọn lửa được đốt lên từ chính ngôi nhà mình. Nam diễn viên diễn chừng mực, không lên gân hay gào thét để người xem tự thấu cảm về nỗi đau cùng cực.
"Do quá nhập vai, tôi đã tự lấy bật lửa đốt tay mình và đến giờ vẫn còn để lại vết bỏng", anh kể. Trước ngày phim bấm máy, nam diễn viên dành một tháng để đến set quay, học nghề ở các lò than gần một tháng để đảm bảo hóa thân tốt. 
"""

text =  sentence + " </s>"
encoding = tokenizer(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=False)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

RuntimeError: ignored